# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

In [1]:
import os
import os.path
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pymysql
import numpy as np
import pickle
from tabulate import tabulate


1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [4]:
with open('addres-book.json', 'r') as f:
    fil_j = json.load(f)
    
#Способ 1
for i in fil_j:
    print(i['email'])

#Способ 2
print(fil_j[0]['email'], fil_j[1]['email'])

faina@mail.ru
robert@mail.ru
faina@mail.ru robert@mail.ru


2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [5]:
#Способ 1
for i in fil_j:
    print(*i['phones'])

#Способ 2
print(*fil_j[0]['phones'], *fil_j[1]['phones'])

{'phone': '232-19-55'} {'phone': '+7 (916) 232-19-55'}
{'phone': '111-19-55'} {'phone': '+7 (916) 445-19-55'}
{'phone': '232-19-55'} {'phone': '+7 (916) 232-19-55'} {'phone': '111-19-55'} {'phone': '+7 (916) 445-19-55'}


3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [6]:
with open('addres-book-q.xml') as f:
    fil_x = BeautifulSoup(f, 'xml')

ph = list()
for pers in fil_x.address_book.find_all("address"):
    ph1 = [phones.next for phones in pers.phones.find_all("phone")]
    ph.append({pers.find("name").next : ph1})
ph

[{'Aicha Barki': ['+ (213) 6150 4015', '+ (213) 2173 5247']},
 {'Francisco Domingos': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Maria Luisa': ['+ (244) 4232 2836']},
 {'Abraao Chanda': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Beatriz Busaniche': ['+ (54-11) 4784 1159']},
 {'Francesca Beddie': ['+ (61-2) 6274 9500', '+ (61-2) 6274 9513']},
 {'Graham John Smith': ['+ (61-3) 9807 4702']}]

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [4]:
with open('contributors_sample.json', 'r') as read_f:
    fil_j = json.load(read_f)
fil_j[0:3]

[{'username': 'uhebert',
  'name': 'Lindsey Nguyen',
  'sex': 'F',
  'address': '01261 Cameron Spring\nTaylorfurt, AK 97791',
  'mail': 'jsalazar@gmail.com',
  'jobs': ['Energy engineer',
   'Engineer, site',
   'Environmental health practitioner',
   'Biomedical scientist',
   'Jewellery designer'],
  'id': 35193},
 {'username': 'vickitaylor',
  'name': 'Cheryl Lewis',
  'sex': 'F',
  'address': '66992 Welch Brooks\nMarshallshire, ID 56004',
  'mail': 'bhudson@gmail.com',
  'jobs': ['Music therapist',
   'Volunteer coordinator',
   'Designer, interior/spatial'],
  'id': 91970},
 {'username': 'sheilaadams',
  'name': 'Julia Allen',
  'sex': 'F',
  'address': 'Unit 1632 Box 2971\nDPO AE 23297',
  'mail': 'darren44@yahoo.com',
  'jobs': ['Management consultant',
   'Engineer, structural',
   'Lecturer, higher education',
   'Theatre manager',
   'Designer, textile'],
  'id': 1848091}]

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [5]:
k = []
for i in fil_j:
    s = i['mail']
    s_pr = s[s.find("@") : ]
    k.append(s_pr)
print(set(k))

{'@hotmail.com', '@gmail.com', '@yahoo.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [6]:
#Без исключения ValueError
k = 0
usn = str(input("Введите 'Username' пользователя(Пример: uhebert): "))
for n in fil_j:
    if usn == n['username']:
        print(n)
        k = 1
if k == 0:
    print("Нет в базе! Введите 'Username' правильно!")


Нет в базе! Введите 'Username' правильно!


In [6]:
#C исключением ValueError
k = 0
usn = str(input("Введите 'Username' пользователя(Пример: uhebert): "))
for n in fil_j:
    if n['username'] == usn:
        print(n)
        k += 1
if k == 0:
    raise ValueError("Нет в базе! Введите 'Username' правильно!")

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [7]:
m = 0
d = 0
for p in fil_j:
    for p1 in p:
        if p[p1] == 'F':
            d += 1
        elif p[p1] == 'M':
            m += 1
print("Женщины: ", d, '\n', "Мужчины: ", m)
    

Женщины:  2136 
 Мужчины:  2064


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [24]:
#Без DataFrame
tabl = pd.read_json('contributors_sample.json')
contributors = tabl.drop(columns=['name', 'address', 'mail', 'jobs'])
contributors

,username,sex,id
0,uhebert,F,35193
1,vickitaylor,F,91970
2,sheilaadams,F,1848091
3,nicole82,F,50969
4,jean67,M,676820
...,...,...,...
4195,stevenspencer,F,423555
4196,rwilliams,M,35251
4197,lmartinez,F,135887
4198,brendahill,M,212714


In [25]:
tabl2 = {'id' : [],
        'username' : [],
        'sex' : []}

for t in fil_j:
    tabl2['id'] += [t['id']]
    tabl2['username'] += [t['username']]
    tabl2['sex'] += [t['sex']]

contributors = pd.DataFrame(tabl2)
contributors       


,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [33]:
recipes = pd.read_csv('recipes_sample.csv')
unif = pd.merge(recipes, contributors, how = 'left', on = 'id')
print(unif.isnull().sum())
unif.head()

name                  0
id                    0
minutes               0
contributor_id        0
submitted             0
n_steps           11190
description         623
n_ingredients      8880
username          29830
sex               29830
dtype: int64


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,username,sex
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,NaN,NaN
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,NaN,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,NaN,NaN
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,NaN,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,NaN,NaN


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`